In [8]:
# Cell 1: Core Components
import open3d as o3d
import numpy as np

DEFAULT_PARAMS = {
    'body_scale': [1.5, 1.0, 0.6],
    'head_scale': [0.7, 0.7, 0.5],
    'leg_radius': 0.15,
    'leg_height': 0.4
}

def create_part(shape, params, position=(0,0,0)):
    """Base shape creation function with extension hooks"""
    # Extension point: Add new shape types here
    mesh = getattr(o3d.geometry.TriangleMesh, f'create_{shape}')(**params)
    mesh.compute_vertex_normals()
    mesh.translate(position)
    return mesh

def apply_wrinkles(mesh, intensity=0.02):
    """Modifier function with extension potential"""
    vertices = np.asarray(mesh.vertices)
    mesh.vertices = o3d.utility.Vector3dVector(vertices + np.random.normal(0, intensity, vertices.shape))
    return mesh

In [9]:
# Enhanced Cell 2: Reasoning System with Detailed Output
class FeatureAnalyzer:
    def __init__(self):
        self.reasoning_log = []
        self.detected_features = []
        self.anatomy_map = {
            'body': {'required': True, 'built': False},
            'head': {'required': True, 'built': False},
            'legs': {'required': True, 'built': False},
            'ears': {'required': False, 'built': False},
            'tail': {'required': False, 'built': False},
            'collar': {'required': False, 'built': False}
        }
    
    def log_step(self, message, indent=0):
        """Log and immediately print messages with indentation"""
        indent_str = '  ' * indent
        formatted = f"{indent_str}• {message}"
        self.reasoning_log.append(formatted)
        print(formatted)
    
    def analyze_description(self, description):
        """Enhanced analysis process with verbose logging"""
        text = description.lower()
        self.detected_features = []
        
        self.log_step(f"Starting analysis of: '{description}'")
        self.log_step("Phase 1: Feature Detection", indent=1)
        
        feature_triggers = {
            'wrinkled': ['wrinkle', 'wrinkled', 'fold'],
            'short_legs': ['short legs', 'stubby legs'],
            'floppy_ears': ['floppy ears', 'droopy ears'],
            'stubby_tail': ['stubby tail', 'short tail'],
            'collar': ['collar', 'neckband', 'leather']
        }
        
        # Feature detection with keyword breakdown
        for feature, keywords in feature_triggers.items():
            self.log_step(f"Checking '{feature}'...", indent=2)
            matches = [kw for kw in keywords if kw in text]
            
            if matches:
                self.detected_features.append(feature)
                self.log_step(f"MATCHED {len(matches)} keywords: {', '.join(matches)}", indent=3)
            else:
                self.log_step(f"No matches from: {', '.join(keywords)}", indent=3)
        
        # Anatomy validation with status report
        self.log_step("\nPhase 2: Anatomy Validation", indent=1)
        for part, info in self.anatomy_map.items():
            status = "✓ Built" if info['built'] else "✗ Missing"
            req_type = "Required" if info['required'] else "Optional"
            self.log_step(f"{part.upper()}: {status} ({req_type})", indent=2)
            
            if info['required'] and not info['built']:
                self.log_step("WARNING: Required component not implemented!", indent=3)
        
        # Final summary
        self.log_step("\nAnalysis Results:", indent=1)
        self.log_step(f"Detected Features: {', '.join(self.detected_features) or 'None'}", indent=2)
        self.log_step(f"Anatomy Complete: {all(info['built'] or not info['required'] for info in self.anatomy_map.values())}", indent=2)
        
        return self.detected_features

# Example usage demonstration
if __name__ == "__main__":
    print("=== Sample Analysis ===")
    analyzer = FeatureAnalyzer()
    sample_description = "A dog with wrinkled skin and a stubby tail"
    features = analyzer.analyze_description(sample_description)

=== Sample Analysis ===
• Starting analysis of: 'A dog with wrinkled skin and a stubby tail'
  • Phase 1: Feature Detection
    • Checking 'wrinkled'...
      • MATCHED 2 keywords: wrinkle, wrinkled
    • Checking 'short_legs'...
      • No matches from: short legs, stubby legs
    • Checking 'floppy_ears'...
      • No matches from: floppy ears, droopy ears
    • Checking 'stubby_tail'...
      • MATCHED 1 keywords: stubby tail
    • Checking 'collar'...
      • No matches from: collar, neckband, leather
  • 
Phase 2: Anatomy Validation
    • BODY: ✗ Missing (Required)
      • WARNING: Required component not implemented!
    • HEAD: ✗ Missing (Required)
      • WARNING: Required component not implemented!
    • LEGS: ✗ Missing (Required)
      • WARNING: Required component not implemented!
    • EARS: ✗ Missing (Optional)
    • TAIL: ✗ Missing (Optional)
    • COLLAR: ✗ Missing (Optional)
  • 
Analysis Results:
    • Detected Features: wrinkled, stubby_tail
    • Anatomy Complete: Fal

In [13]:
# Cell 4: Full Bulldog Generator Implementation
class BulldogGenerator:
    def __init__(self):
        self.components = []
        self.analyzer = FeatureAnalyzer()
        
        # Ensure all required parameters exist
        self.params = {
            **DEFAULT_PARAMS,
            'tail_radius': 0.1,
            'collar_radius': 0.4
        }

    def _apply_scaling(self, mesh, scale_factors):
        """Scaling method implementation"""
        scaling_matrix = np.eye(4)
        scaling_matrix[0, 0] = scale_factors[0]
        scaling_matrix[1, 1] = scale_factors[1]
        scaling_matrix[2, 2] = scale_factors[2]
        mesh.transform(scaling_matrix)
        return mesh

    def _add_body(self, features):
        self.analyzer.log_step("Building body...", indent=1)
        body = create_part('sphere', {'radius': 1})
        self._apply_scaling(body, self.params['body_scale'])
        self.components.append(body)
        self.analyzer.anatomy_map['body']['built'] = True

    def _add_legs(self):
        self.analyzer.log_step("Adding legs...", indent=1)
        body_depth = self.params['body_scale'][2]
        positions = [
            (0.4, 0.4, -body_depth),
            (-0.4, 0.4, -body_depth),
            (0.4, -0.4, -body_depth),
            (-0.4, -0.4, -body_depth)
        ]
        
        for pos in positions:
            leg = create_part('cylinder', {
                'radius': self.params['leg_radius'],
                'height': self.params['leg_height']
            }, pos)
            self.components.append(leg)
            
            # Add paw with proper syntax
            paw = create_part('sphere', {
                'radius': self.params['leg_radius']*1.2
            }, (pos[0], pos[1], pos[2] - self.params['leg_height']/2))
            self.components.append(paw)
            
        self.analyzer.anatomy_map['legs']['built'] = True

    def _add_face(self, features):
        self.analyzer.log_step("Creating face...", indent=1)
        # Add basic face components
        body_width = self.params['body_scale'][0]
        head_pos = (body_width/2 + 0.3, 0, 0.2)
        
        # Add nose
        nose = create_part('cylinder', 
            {'radius': 0.1, 'height': 0.05},
            (head_pos[0], 0, head_pos[2] + 0.1))
        self.components.append(nose)
        
    def _add_tail(self, features):
        if 'stubby_tail' in features:
            self.analyzer.log_step("Adding tail...", indent=1)
            tail_pos = (-self.params['body_scale'][0]/2 - 0.1, 0, 0.1)
            tail = create_part('cone', {
                'radius': self.params['tail_radius'],
                'height': 0.2
            }, tail_pos)
            self.components.append(tail)
            self.analyzer.anatomy_map['tail']['built'] = True

    def generate(self, description):
        self.components = []
        features = self.analyzer.analyze_description(description)
        
        # Build sequence
        self._add_body(features)
        self._add_legs()
        self._add_face(features)
        self._add_tail(features)
        
        # Merge components
        full_mesh = self.components[0]
        for part in self.components[1:]:
            full_mesh += part
            
        return full_mesh

# Usage test
generator = BulldogGenerator()
bulldog = generator.generate("Wrinkled bulldog with stubby tail")
print("Generation successful!")

# Cell 5: Critique Agent Implementation
class BulldogCritic:
    def __init__(self, generator):
        self.generator = generator
        self.critique_log = []
        
    def analyze(self):
        """Basic anatomy validation"""
        missing = [
            part for part, specs in self.generator.analyzer.anatomy_map.items()
            if specs['required'] and not specs['built']
        ]
        if missing:
            self.critique_log.append(f"Missing required components: {', '.join(missing)}")

    def get_report(self):
        return "\n".join(self.critique_log) or "No critical issues found"

# Proper usage
generator = BulldogGenerator()
bulldog = generator.generate("Wrinkled bulldog with stubby tail")
critic = BulldogCritic(generator)
critic.analyze()
print("Generation Report:")
print(critic.get_report())

• Starting analysis of: 'Wrinkled bulldog with stubby tail'
  • Phase 1: Feature Detection
    • Checking 'wrinkled'...
      • MATCHED 2 keywords: wrinkle, wrinkled
    • Checking 'short_legs'...
      • No matches from: short legs, stubby legs
    • Checking 'floppy_ears'...
      • No matches from: floppy ears, droopy ears
    • Checking 'stubby_tail'...
      • MATCHED 1 keywords: stubby tail
    • Checking 'collar'...
      • No matches from: collar, neckband, leather
  • 
Phase 2: Anatomy Validation
    • BODY: ✗ Missing (Required)
      • WARNING: Required component not implemented!
    • HEAD: ✗ Missing (Required)
      • WARNING: Required component not implemented!
    • LEGS: ✗ Missing (Required)
      • WARNING: Required component not implemented!
    • EARS: ✗ Missing (Optional)
    • TAIL: ✗ Missing (Optional)
    • COLLAR: ✗ Missing (Optional)
  • 
Analysis Results:
    • Detected Features: wrinkled, stubby_tail
    • Anatomy Complete: False
  • Building body...
  • Addin

In [11]:
# Cell 5: Critique Agent
class BulldogCritic:
    def __init__(self, generator):
        self.generator = generator
        self.critique_log = []
        self.recommendations = []
    
    def _log_issue(self, issue, severity="MEDIUM"):
        self.critique_log.append({
            'issue': issue,
            'severity': severity,
            'component': None
        })
    
    def analyze_workflow(self):
        # Anatomy completeness check
        missing_core = [p for p,v in self.generator.analyzer.anatomy_map.items() 
                       if v['required'] and not v['built']]
        if missing_core:
            self._log_issue(f"Missing core components: {', '.join(missing_core)}", "HIGH")
        
        # Bulldog-specific feature check
        bulldog_requirements = {
            'head': ['flat_face', 'pronounced_jowls'],
            'body': ['muscular_chest', 'wide_shoulders'],
            'legs': ['stout_proportions', 'front_heavy']
        }
        
        # Construction order analysis
        build_steps = [log for log in self.generator.analyzer.reasoning_log 
                      if "Building:" in log]
        
        # Parameter validation
        body_scale = DEFAULT_PARAMS['body_scale']
        if body_scale[0]/body_scale[1] < 1.4:
            self.recommendations.append("Increase body width-to-height ratio for bulldog profile")
            
    def generate_report(self):
        report = [
            "Bulldog Generation Critique Report",
            "="*40,
            f"Critical Issues: {sum(1 for i in self.critique_log if i['severity'] == 'HIGH')}",
            f"Recommendations: {len(self.recommendations)}",
            "\nTop Improvements:"
        ]
        
        return '\n'.join(report + self.recommendations[:3])

# Usage
generator = BulldogGenerator()
bulldog = generator.generate("Wrinkled bulldog with stubby tail")
critic = BulldogCritic(generator)
critic.analyze_workflow()
print(critic.generate_report())

AttributeError: 'BulldogGenerator' object has no attribute 'generate'

In [ ]:
# no good
import open3d as o3d
import numpy as np

class BulldogGenerator:
    def __init__(self):
        self.components = []
        self.defaults = {
            'body_scale': [1.5, 1.0, 0.6],
            'head_scale': [0.7, 0.7, 0.5],
            'leg_radius': 0.15,
            'leg_height': 0.4
        }

    def _create_part(self, shape, params, position):
        mesh = getattr(o3d.geometry.TriangleMesh, f'create_{shape}')(**params)
        mesh.compute_vertex_normals()
        mesh.translate(position)
        return mesh

    def _add_body(self, features):
        body = self._create_part('sphere', {'radius': 1}, (0,0,0))
        body.scale(self.defaults['body_scale'], center=(0,0,0))
        self.components.append(body)
        
        # Head
        head_pos = (self.defaults['body_scale'][0]/2 + 0.3, 0, 0)
        head = self._create_part('sphere', {'radius': 0.7}, head_pos)
        head.scale(self.defaults['head_scale'], center=head_pos)
        self.components.append(head)
        
        if 'wrinkled' in features:
            vertices = np.asarray(head.vertices)
            head.vertices = o3d.utility.Vector3dVector(vertices + np.random.normal(0, 0.02, vertices.shape))

    def _add_legs(self):
        positions = [
            (0.4, 0.4, -self.defaults['body_scale'][2]),
            (-0.4, 0.4, -self.defaults['body_scale'][2]),
            (0.4, -0.4, -self.defaults['body_scale'][2]),
            (-0.4, -0.4, -self.defaults['body_scale'][2])
        ]
        for pos in positions:
            leg = self._create_part('cylinder', 
                {'radius': self.defaults['leg_radius'], 'height': self.defaults['leg_height']}, 
                pos
            )
            self.components.append(leg)

    def _add_face(self):
        # Simple eyes
        for x in [0.2, -0.2]:
            eye = self._create_part('sphere', {'radius': 0.08}, 
                (self.defaults['body_scale'][0]/2 + 0.3 + x, 0.15, 0.1))
            self.components.append(eye)

    def generate(self, description=""):
        features = [f for f in ['wrinkled', 'short', 'collar'] if f in description.lower()]
        
        self._add_body(features)
        self._add_legs()
        self._add_face()
        
        # Combine all parts
        full_mesh = self.components[0]
        for part in self.components[1:]:
            full_mesh += part
            
        return full_mesh

# Usage
generator = BulldogGenerator()
bulldog = generator.generate("wrinkled short bulldog")
o3d.visualization.draw_geometries([bulldog])

TypeError: scale(): incompatible function arguments. The following argument types are supported:
    1. (self: open3d.cpu.pybind.geometry.Geometry3D, scale: float, center: numpy.ndarray[numpy.float64[3, 1]]) -> open3d.cpu.pybind.geometry.Geometry3D
    2. (self: open3d.cpu.pybind.geometry.Geometry3D, scale: float, center: numpy.ndarray[numpy.float64[3, 1]]) -> open3d.cpu.pybind.geometry.Geometry3D

Invoked with: TriangleMesh with 762 points and 1520 triangles., [1.5, 1.0, 0.6]; kwargs: center=(0, 0, 0)